In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/googlenewsvectorsnegative300/GoogleNews-vectors-negative300.bin
/kaggle/input/train-test-data/test.csv
/kaggle/input/train-test-data/train.csv


In [3]:
import pandas as pd
test = pd.read_csv("../input/train-test-data/test.csv")
train = pd.read_csv("../input/train-test-data/train.csv")

In [4]:
train[:2]

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0


In [5]:
import math
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split


In [6]:
train_df,val_df =train_test_split(train,test_size=0.1)

In [7]:
from gensim.models import KeyedVectors
news_path="/kaggle/input/googlenewsvectorsnegative300/GoogleNews-vectors-negative300.bin"
embeddings_index=KeyedVectors.load_word2vec_format(news_path,binary=True)

In [8]:
def retval(x):
    empty_emb=np.zeros(300)
    try: 
        return embeddings_index[x]
    except:
        return empty_emb
def text_to_array(text):
    empty_emb=np.zeros(300)
    text = text[:-1].split()[:30]
    embeds = [retval(x) for x in text]
    embeds+=[empty_emb]*(30-len(embeds))
    return np.array(embeds)

val_vects=np.array([text_to_array(X_test) for X_test in val_df["question_text"][:3000]])
val_y=np.array(val_df["target"][:3000])

In [9]:
val_vects.shape

(3000, 30, 300)

In [10]:
batch_size = 128

def batch_gen(train_df):
    n_batches = math.ceil(len(train_df) / batch_size)
    while True:
        train_df = train_df.sample(frac=1.)  # Shuffle the data.
        for i in range(n_batches):
            texts = train_df.iloc[i*batch_size:(i+1)*batch_size, 1]
            text_arr = np.array([text_to_array(text) for text in texts])
            yield text_arr, np.array(train_df["target"][i*batch_size:(i+1)*batch_size])

In [11]:
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.layers import CuDNNLSTM, Dense, Bidirectional

Using TensorFlow backend.


In [12]:
mg = batch_gen(train_df)

In [13]:
mg

<generator object batch_gen at 0x7f12ceaf3888>

In [14]:
import tensorflow as tf
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import Dropout
from keras.preprocessing.text import Tokenizer
# model = Sequential()
# inp=tf.keras.layers.Input(shape=(30,300))
# x2=tf.keras.layers.Bidirectional(tf.compat.v1.keras.layers.CuDNNGRU(64, return_sequences=True))(inp)
# # model.add(Bidirectional(CuDNNLSTM(64, return_sequences=True),input_shape=(30, 300)))
# # model.add(Bidirectional(CuDNNLSTM(64)))
# max_pl = tf.keras.layers.GlobalMaxPooling1D()(x2)
# model.add(Dense(1, activation="sigmoid"))

# model.compile(loss='binary_crossentropy',
#               optimizer='adam',
#               metrics=['accuracy'])
model = Sequential()
model.add(LSTM(100, input_shape=(30,300)))
model.add(Dropout(0.2))
#    model.add(LSTM(70))
#    model.add(Dropout(0.3))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
# inp = tf.keras.layers.Input(shape=(10,))
# emb = tf.keras.layers.Embedding(20, 4)(inp)
# x2 = tf.keras.layers.Bidirectional(tf.compat.v1.keras.layers.CuDNNGRU(64, return_sequences=True))(emb)
# max_pl = tf.keras.layers.GlobalMaxPooling1D()(x2)
# x = tf.keras.layers.Dense(16, activation="relu")(max_pl)
# x = tf.keras.layers.Dropout(0.1)(x)
# output = tf.keras.layers.Dense(1, activation="sigmoid")(x)

# model = tf.keras.models.Model(inputs=inp, outputs=output)
# model.summary()



In [15]:
model.fit_generator(mg, epochs=20,
                    steps_per_epoch=1000,
                    validation_data=(val_vects, val_y),
                    verbose=True)

Epoch 1/20
1000/1000 [==============================] - 51s 51ms/step - loss: 0.0414 - val_loss: 0.0358
Epoch 2/20
1000/1000 [==============================] - 48s 48ms/step - loss: 0.0375 - val_loss: 0.0341
Epoch 3/20
1000/1000 [==============================] - 48s 48ms/step - loss: 0.0357 - val_loss: 0.0314
Epoch 4/20
1000/1000 [==============================] - 48s 48ms/step - loss: 0.0345 - val_loss: 0.0315
Epoch 5/20
1000/1000 [==============================] - 47s 47ms/step - loss: 0.0337 - val_loss: 0.0320
Epoch 6/20
1000/1000 [==============================] - 47s 47ms/step - loss: 0.0338 - val_loss: 0.0301
Epoch 7/20
1000/1000 [==============================] - 47s 47ms/step - loss: 0.0327 - val_loss: 0.0300
Epoch 8/20
1000/1000 [==============================] - 47s 47ms/step - loss: 0.0334 - val_loss: 0.0300
Epoch 9/20
1000/1000 [==============================] - 48s 48ms/step - loss: 0.0330 - val_loss: 0.0301
Epoch 10/20
1000/1000 [==============================] - 46s 46m

In [50]:
model.predict(np.array([text_to_array(x) for x in test["question_text"][21:23]]))

array([[0.7413736]], dtype=float32)

In [51]:
test["question_text"][21]

'Why do so many people on this damn site not actually answer the question, but give some stupid and probably fake anecdotes instead?'